![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [8]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV

filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features = features.squeeze('columns')
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_train = subject_train.squeeze('columns')
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
subject_test = subject_test.squeeze('columns')
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=23)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [86]:
%%time

clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=23).fit(X_train, y_train)

CPU times: total: 4.05 s
Wall time: 5.58 s


In [79]:
def avalia (model: pd.DataFrame, conjunto_X: list, conjunto_y:list, nomes=['treino', 'validação', 'teste'], exibir = True):
    resultados = {}
    for nome, X, y in zip(nomes, conjunto_X, conjunto_y):
        scores = model.score(X, y)
        resultados[nome]= scores
        if exibir:
            print(f'Acurácia na base de {nome}: {scores*100:.1f}' )
    return resultados

In [87]:
resultado = avalia(clf,[X_train, X_valid, X_test], [y_train, y_valid, y_test])

Acurácia na base de treino: 97.8
Acurácia na base de validação: 94.3
Acurácia na base de teste: 86.6


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [81]:
%%time
prcomp = PCA().fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)

pc_treino.shape

CPU times: total: 1.5 s
Wall time: 1.2 s


(5514, 561)

In [82]:
n=1

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_valid = pd.DataFrame(pc_valida[:,:n], columns = colunas)
pc_test  = pd.DataFrame(pc_teste[:,:n], columns = colunas)

pc_train.head()

,cp1
0,-6.121259
1,-5.184451
2,-2.473830
3,-5.826141
4,-5.799838


In [88]:
%%time
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=23).fit(pc_train, y_train)

CPU times: total: 62.5 ms
Wall time: 69.5 ms


In [89]:
acuracia = avalia(clf, [pc_train, pc_valid, pc_test], [y_train, y_valid, y_test])

Acurácia na base de treino: 50.2
Acurácia na base de validação: 49.1
Acurácia na base de teste: 45.4


A acurácia diminuiu em relação a primeira árvore treinada com todas as componentes.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [95]:
%%time
comp = [1, 2, 5, 10, 25, 50, 100, 150, 200]
bases = {}

for n in comp:
    pca = PCA(n_components=n)
    pca.fit(X_train)
    
    pca_treino = pca.transform(X_train)
    pca_valida = pca.transform(X_valid)
    pca_teste = pca.transform(X_test)
    
    colunas = [f'cp{i+1}' for i in range(n)]
    
    bases[f'pca_train_{n}'] = pd.DataFrame(pca_treino, columns=colunas)
    bases[f'pca_valid_{n}'] = pd.DataFrame(pca_valida, columns=colunas)
    bases[f'pca_test_{n}'] = pd.DataFrame(pca_teste, columns=colunas)

CPU times: total: 8.12 s
Wall time: 5.26 s


Visualização das bases criadas:

In [96]:
for chave, df in bases.items():
    if 'pca_train' in chave:
        print(f'\n{chave}:')
        display(df.head())


pca_train_1:


,cp1
0,-6.121259
1,-5.184451
2,-2.473830
3,-5.826141
4,-5.799838



pca_train_2:


,cp1,cp2
0,-6.121259,2.296441
1,-5.184451,-0.712840
2,-2.473830,-2.994641
3,-5.826141,0.308928
4,-5.799838,1.320340



pca_train_5:


...

,cp1,cp2,cp3,cp4,cp5,cp6,cp7,cp8,cp9,cp10,...,cp91,cp92,cp93,cp94,cp95,cp96,cp97,cp98,cp99,cp100
0,-6.121259,2.296441,-2.146628,0.572644,0.106552,-1.008276,-0.245330,-0.322491,-0.532149,0.073749,...,0.181175,0.157361,0.175665,0.172103,0.004488,0.050417,-0.000719,0.130851,-0.016783,0.116808
1,-5.184451,-0.712840,-1.388140,0.420122,0.625450,0.651448,-0.501937,0.303652,0.383500,0.181302,...,-0.152057,-0.072826,-0.230138,-0.146181,-0.121885,0.180631,0.087919,0.130696,0.076996,-0.072317
2,-2.473830,-2.994641,3.373997,0.373222,1.847946,1.265870,1.169315,-0.516178,-0.544651,-0.861302,...,-0.000860,0.095230,0.418222,-0.204705,-0.241493,0.442552,-0.426342,0.057879,0.393264,-0.015882
3,-5.826141,0.308928,2.472890,0.398020,0.381351,0.204011,0.842363,0.231420,-0.716721,0.291936,...,-0.016883,0.129614,0.222267,-0.045908,0.131979,-0.085853,0.275211,0.142921,-0.232475,-0.354718
4,-5.799838,1.320340,-1.732920,0.075927,-0.433481,-0.358010,-0.693687,0.515134,0.413586,0.250095,...,-0.249886,0.084984,-0.089368,0.016256,0.131160,-0.109179,0.144886,-0.108467,0.000577,-0.279894



pca_train_150:


,cp1,cp2,cp3,cp4,cp5,cp6,cp7,cp8,cp9,cp10,...,cp141,cp142,cp143,cp144,cp145,cp146,cp147,cp148,cp149,cp150
0,-6.121259,2.296441,-2.146628,0.572644,0.106552,-1.008276,-0.245330,-0.322491,-0.532149,0.073749,...,0.005718,-0.083940,0.001358,0.073941,0.064888,-0.052149,0.051819,0.066455,0.042022,0.032921
1,-5.184451,-0.712840,-1.388140,0.420122,0.625450,0.651448,-0.501937,0.303652,0.383500,0.181302,...,0.026033,-0.056348,-0.077746,-0.060197,0.049168,-0.074898,-0.053926,-0.077322,-0.044596,0.014680
2,-2.473830,-2.994641,3.373997,0.373222,1.847946,1.265870,1.169315,-0.516178,-0.544651,-0.861301,...,0.026585,-0.146141,-0.106934,0.108006,-0.094004,-0.263653,-0.270327,-0.042242,0.178139,-0.148572
3,-5.826141,0.308928,2.472890,0.398020,0.381351,0.204011,0.842363,0.231420,-0.716721,0.291936,...,-0.092757,0.011628,0.076344,-0.005688,0.062772,-0.008777,-0.099493,0.058262,0.003344,0.030763
4,-5.799838,1.320340,-1.732920,0.075927,-0.433481,-0.358010,-0.693687,0.515134,0.413586,0.250095,...,-0.004354,0.015963,0.134380,-0.021854,0.009291,0.019472,-0.034310,0.096440,-0.133192,0.041774



pca_train_200:


,cp1,cp2,cp3,cp4,cp5,cp6,cp7,cp8,cp9,cp10,...,cp191,cp192,cp193,cp194,cp195,cp196,cp197,cp198,cp199,cp200
0,-6.121259,2.296441,-2.146628,0.572644,0.106552,-1.008276,-0.245330,-0.322491,-0.532149,0.073749,...,0.020338,0.015389,-0.014784,0.042300,-0.020025,0.009787,0.035340,-0.004165,-0.018027,-0.027548
1,-5.184451,-0.712840,-1.388140,0.420122,0.625450,0.651448,-0.501937,0.303652,0.383500,0.181302,...,-0.036270,0.033391,0.031197,-0.040088,0.075635,0.003599,-0.013064,0.013236,0.024967,0.014827
2,-2.473830,-2.994641,3.373997,0.373222,1.847946,1.265870,1.169315,-0.516178,-0.544651,-0.861301,...,0.136822,-0.031469,-0.027008,0.085444,0.096142,-0.025736,-0.078698,0.148236,0.152459,0.291425
3,-5.826141,0.308928,2.472890,0.398020,0.381351,0.204011,0.842363,0.231420,-0.716721,0.291936,...,0.006961,-0.034690,-0.022807,-0.024974,0.007982,-0.009884,0.015749,-0.033388,-0.032038,0.024630
4,-5.799838,1.320340,-1.732920,0.075927,-0.433481,-0.358010,-0.693687,0.515134,0.413586,0.250095,...,-0.016742,0.005988,-0.008124,-0.011500,-0.011013,-0.006866,-0.009906,-0.000341,0.060509,-0.008085


In [103]:
%%time
resultado_acc = {}
for n in comp:
    
    X_treino = bases[f'pca_train_{n}']
    X_valida = bases[f'pca_valid_{n}']
    X_teste = bases[f'pca_test_{n}']
    
    clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=23).fit(X_treino, y_train)
    
    avaliacao = avalia(clf, [X_treino, X_valida, X_teste], [y_train, y_valid, y_test], exibir=False)
    
    resultado_acc[f'{n}_comp'] = avaliacao

CPU times: total: 6.92 s
Wall time: 7.19 s


In [104]:
score = pd.DataFrame(resultado_acc).T

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

In [105]:
score

,treino,validação,teste
1_comp,0.501632,0.490751,0.454021
2_comp,0.617338,0.595212,0.578554
5_comp,0.850925,0.831338,0.783509
10_comp,0.895357,0.858542,0.808280
25_comp,0.912768,0.869423,0.799796
50_comp,0.931266,0.861806,0.803529
100_comp,0.937976,0.877040,0.796742
150_comp,0.941603,0.873776,0.799457
200_comp,0.941059,0.872144,0.791992


É possível observar o fenômeno de Huges acontecendo. Conforme o número de componentes aumenta o valor da acurácia cai, tanto para a base de validação quanto a base de teste.

Quanto ao tempo, a árvore treinada com todas as componentes demorou 4 segundos, enquanto a árvore treinada apenas com 1 componente demorou 62 milisegundos. Já o treinamento das árvores com os componentes 1, 2, 5, 10, 25, 50, 100, 150 e 200 demorou apenas 7 segundos. O que mostra a eficiência da utilização do PCA.